1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이스베이스에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달

In [1]:
%pip install -qU pypdf langchain-community


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU langchain-text-splitters


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install chromadb
%pip install langchain
%pip install sentence-transformers
%pip install docx2txt


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -qU langchain langchainhub 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

# 문서 불러오기
#file_path = "text.pdf"
#loader = PyPDFLoader(file_path)
#pages = loader.load()  # 기본 로드 방식 사용

# 문서 쪼개기
# chunk_size: 문서를 쪼개는 단위
# chunk_overlap: 문서를 쪼개는 단위의 겹치는 부분
# 겹치는 부분을 선언하는 이유는 원하는 문서를 가져올 확률을 높이기 위함
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
#chunks = text_splitter.split_documents(pages)

# load_and_split 사용으로 문서 불러오기와 쪼개기를 한 번에 수행
# 분할기 정의
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

# 문서 불러오기와 쪼개기를 한 번에 수행
# 기존 docx에 소득세 정보는 사진의 표로 들어가 있기 때문에 gpt가 찾지 못한다.
# 따라서 docx의 내용을 markdown으로 변환하여 저장해주어야 한다.
file_path = "text.docx"
loader = Docx2txtLoader(file_path)
document_list = loader.load_and_split(text_splitter)


# 임베딩
load_dotenv()
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [6]:
%pip install -qU langchain-pinecone 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
from dotenv import load_dotenv

from langchain_pinecone import PineconeVectorStore

load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embeddings,
    index_name="tax-markdown-index",
    pinecone_api_key=pinecone_api_key
)



In [8]:
# 유사도 검색
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [9]:
retrieved_docs

[]

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

# 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달
# 페르소나를 전달하면 답변이 더 잘나온다고 한다.
prompt = f"""[Identity]
- 당신은 한국의 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 대답해주세요.

[Context]
{retrieved_docs}

[Question]
{query}
"""
response = llm.invoke(prompt)

In [11]:
response.content

'연봉 3,000만 원인 직장인의 소득세를 계산하기 위해서는 특정한 공제를 고려하지 않는 기본적인 계산을 해보겠습니다. 대한민국의 소득세는 누진세로, 소득 구간별로 다른 세율이 적용됩니다. 2023년 기준으로 소득세율은 다음과 같습니다.\n\n1. **1,200만 원 이하**: 6%\n2. **1,200만 원 초과 ~ 4,600만 원 이하**: 15%\n3. **4,600만 원 초과 ~ 8,800만 원 이하**: 24%\n4. **8,800만 원 초과 ~ 1억 5,000만 원 이하**: 35%\n5. **1억 5,000만 원 초과 ~ 3억 원 이하**: 38%\n6. **3억 원 초과 ~ 5억 원 이하**: 40%\n7. **5억 원 초과 ~ 10억 원 이하**: 42%\n8. **10억 원 초과**: 45%\n\n연봉 3,000만 원의 경우를 계산해보면:\n\n- 1,200만 원까지는 6%: 1,200만 원 × 6% = 72만 원\n- 나머지 1,800만 원(3,000만 원 - 1,200만 원)에 대해 15%: 1,800만 원 × 15% = 270만 원\n\n따라서 소득세 총액은 72만 원 + 270만 원 = 342만 원입니다.\n\n위의 계산은 기본적인 소득세 계산이며, 실제 소득세 계산 시에는 다양한 공제 항목(인적 공제, 특별 공제 등)과 주민세(소득세의 10%)가 포함될 수 있으므로, 실제 납부해야 할 금액은 달라질 수 있습니다. 보다 정확한 금액을 원하신다면 공제 내역 등을 고려하여 세무 전문가와 상담하시거나 국세청 홈택스의 종합소득세 계산기를 활용하시기 바랍니다.'

In [12]:
from langchain import hub

# question-answering prompt
# 기존에 이미 있는 프롬프트 활용
prompt = hub.pull("rlm/rag-prompt")

/Users/jys/code/study/study-llm-application/venv310/lib/python3.10/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [14]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)


In [15]:
ai_message = qa_chain.invoke({"query": query})

In [16]:
ai_message

{'query': '연봉 3000만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 3000만원인 직장인의 소득세는, 연간 소득에서 1,400만원을 초과하는 금액에 대해 15%의 세율이 적용됩니다. 우선 1,400만원에 대해 6%의 세율이 적용되어 84만원을 계산하고, 1,600만원(3,000만원 - 1,400만원)에 대해 15%의 세율을 적용하면 240만원입니다. 그러므로 총 소득세는 324만원(84만원 + 240만원)입니다.'}